In [1]:
from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget
import numpy as np
import tensorflow as tf
import time

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 22*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

import sys
import os
current = os.getcwd()
parent_directory = os.path.dirname(current)
sys.path.append(parent_directory)
from ICET_spherical import ICET
from utils import R_tf
from metpy.calc import lat_lon_grid_deltas
from scipy.spatial.transform import Rotation as R
from matplotlib import pyplot as plt
import copy
import trimesh
%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook


2023-10-21 16:07:12.240231: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-21 16:07:12.871424: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-21 16:07:13.989613: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-21 16:07:13.989710: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-10-21 16:07:16.294331: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-21 16:07:16.296132: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-21 16:07:16.296411: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-21 16:07:16.296610: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Autosaving every 180 seconds


In [2]:
# #test loading individual subsections
# # pl = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-2ndSection - mesh.ply'
# pl = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-3rdSection.ply'
# HD_map = trimesh.load(pl).vertices
# show_nth = 10 #10
# submap = HD_map[::show_nth]
# plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
# disp=[]
# disp.append(Points(submap, c = "#CB2314", r = 2, alpha = 0.1)) 
# plt.show(disp, "HD Map")
# ViewInteractiveWidget(plt.window)

In [3]:
#stitch together full HD Map
show_nth = 20 #10
disp=[]
plt2 = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)

pl1 = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-1stSection - mesh.ply'
HD_map1 = trimesh.load(pl1).vertices
submap1 = HD_map1[::show_nth]
print(np.shape(submap1))

pl2 = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-2ndSection.ply'
HD_map2 = trimesh.load(pl2).vertices
submap2 = HD_map2[::show_nth]
print(np.shape(submap2))

pl3 = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-3rdSection.ply'
HD_map3 = trimesh.load(pl3).vertices
submap3 = HD_map3[::show_nth]
print(np.shape(submap3))

pl4 = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-4thSection.ply'
HD_map4 = trimesh.load(pl4).vertices
submap4 = HD_map4[::show_nth]
print(np.shape(submap4))

pl5 = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-5thSection.ply'
HD_map5 = trimesh.load(pl5).vertices
submap5 = HD_map5[::show_nth]
print(np.shape(submap5))

# disp.append(Points(submap1, c = "red", r = 2, alpha = 0.1)) 
# disp.append(Points(submap2, c = "orange", r = 2, alpha = 0.1)) 
# disp.append(Points(submap3, c = "yellow", r = 2, alpha = 0.1)) 
# disp.append(Points(submap4, c = "green", r = 2, alpha = 0.1)) 
# disp.append(Points(submap5, c = "blue", r = 2, alpha = 0.1)) 
disp.append(Points(submap1, c = "grey", r = 2, alpha = 0.1)) 
disp.append(Points(submap2, c = "grey", r = 2, alpha = 0.1)) 
disp.append(Points(submap3, c = "grey", r = 2, alpha = 0.1)) 
disp.append(Points(submap4, c = "grey", r = 2, alpha = 0.1)) 
disp.append(Points(submap5, c = "grey", r = 2, alpha = 0.1)) 

plt2.show(disp, "Full HD Map")
ViewInteractiveWidget(plt2.window)

(201237, 3)
(568783, 3)
(837856, 3)
(1575765, 3)
(1060626, 3)


ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

In [4]:
#load ground truth
# [sec,nsec,x,y,z,qx,qy,qz,qw]
fn_gt = "/media/derm/06EF-127D4/Newer College Dataset/01_Short_Experiment/registered_poses.csv"
gt = np.loadtxt(fn_gt, delimiter=',',skiprows = 1)
seconds = gt[:, 0]
nano_seconds = gt[:, 1]
xyz = gt[:, 2:5]
qxyzw = gt[:, 5:]
num_poses = qxyzw.shape[0]
poses = np.eye(4, dtype=np.float64).reshape(1, 4, 4).repeat(num_poses, axis=0)
poses[:, :3, :3] = R.from_quat(qxyzw).as_matrix()
poses[:, :3, 3] = xyz
T_CL = np.eye(4, dtype=np.float32)
T_CL[:3, :3] = R.from_quat([0.0, 0.0, 0.924, 0.383]).as_matrix() #was this --1134.97 deg
T_CL[:3, 3] = np.array([-0.084, -0.025, 0.050], dtype=np.float32) #was this
poses = np.einsum("nij,jk->nik", poses, T_CL)
initial_pose = np.linalg.inv(poses[0]) 
poses_timestamps = seconds * 10e9 + nano_seconds
poses = np.einsum("ij,njk->nik", np.linalg.inv(poses[0]), poses)


#draw GT trajectory
start_idx = 0
gt_lidarframe = gt[:,2:5]
gt_lidarframe = (poses[start_idx] @ np.append(gt_lidarframe, np.ones([len(gt_lidarframe),1]), axis = 1).T).T
# gt_lidarframe[:,:3] -= gt_lidarframe[start_idx,:3]
#rotate 180 deg about vertical axis to align with LOAM results
# gt_lidarframe[:,:3] = gt_lidarframe[:,:3] @ R.from_euler('xyz', [np.deg2rad(0), np.deg2rad(10.25), np.deg2rad(75.5)]).as_matrix() #test
gt_points = Line(gt_lidarframe[:,:3], c = "red", lw = 4, alpha = 1).legend("Actual Motion of Platform")
disp.append(gt_points)

plt3 = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
plt3.show(disp, "Full HD Map")
ViewInteractiveWidget(plt3.window)

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

In [5]:
#plot just trajectory --> let's find a good test trajectory
disp =[]
test_traj = gt[:3000,2:5]
# test_traj_points = Line(test_traj, c = "red", lw = 4, alpha = 1).legend("Actual Motion of Platform")
test_traj_points = Points(test_traj, c = "red", alpha = 1).legend("Actual Motion of Platform")
disp.append(test_traj_points)

plt4 = Plotter(N = 1, axes = 1, bg = (1, 1, 1), interactive = True)
plt4.show(disp, "test trajectory")
ViewInteractiveWidget(plt4.window)

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

In [155]:
#put HD Map and new scan in frame of raw keyframe scan
idx = 5500 #test
skip = 25 #10 #how many lidar frames between keyframe and new scan
offset = 0 #no offset here! This makes life sooo much easier!!!
fn1 = "/media/derm/06EF-127D4/Newer College Dataset/01_Short_Experiment/point_clouds/frame_" + str(idx + offset) + ".npy"
fn2 = "/media/derm/06EF-127D4/Newer College Dataset/01_Short_Experiment/point_clouds/frame_" + str(idx + skip + offset) + ".npy"
pc1 = np.load(fn1)
pc2 = np.load(fn2)
plt = Plotter(N = 1, axes = 1, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

submap = np.concatenate((submap1, submap2, submap3, submap4, submap5), axis = 0) #full scene
# submap = np.concatenate((submap2, submap3, submap4), axis = 0) #test
# submap = submap4 #test
print(np.shape(submap))

#transform everything to pc1 frame
pc2_in_pc1_frame = (np.linalg.pinv(poses[idx]) @ poses[idx+skip] @ np.append(pc2, np.ones([len(pc2),1]), axis=1).T).T
pc2_in_pc1_frame = pc2_in_pc1_frame[:,:3] #remove extra axis
submap_in_pc1_frame = (np.linalg.pinv(poses[idx]) @ initial_pose @ np.append(submap, np.ones([len(submap),1]), axis =1).T).T
submap_in_pc1_frame = submap_in_pc1_frame[:,:3]

# debug: try downsampling submap based on how far away each point is from ego platform
rads = np.sqrt(np.sum(submap_in_pc1_frame**2, axis = 1))
# submap_in_pc1_frame = submap_in_pc1_frame[np.argwhere(rads < 50)[:,0]] # simple distance cutoff (bad)
#randomly drop points proportional to their distance from origin ~~~~~~~
rel_rads = rads/max(rads)
randy = np.random.rand(len(rel_rads))
# submap_in_pc1_frame = submap_in_pc1_frame[np.where(randy + rel_rads**(1/3) < 1)]
submap_in_pc1_frame = submap_in_pc1_frame[np.where(randy + rel_rads**(1/2) < 0.8)]
print(np.shape(submap_in_pc1_frame))
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#remove points from HD Map far outside FOV of sensor
phis = LC.c2s(LC, submap_in_pc1_frame)[:,2]
not_too_low = np.argwhere(phis > 3*np.pi/8)[:,0]
# print(np.shape(not_too_low))
not_too_high = np.argwhere(phis < 5*np.pi/8)[:,0]
# print(len(not_too_high))
good_phis = np.intersect1d(not_too_high, not_too_low)
# print(len(good_phis))
submap_in_pc1_frame = submap_in_pc1_frame[good_phis,:]

# DRAW SUBMAP and SCANS 1, 2
print(np.shape(submap_in_pc1_frame))
disp.append(Points(submap_in_pc1_frame, c = "black", r = 2, alpha = 0.3)) ##CB2314
disp.append(Points(pc1, c = '#a65852', r = 3)) #red
# disp.append(Points(pc2_in_pc1_frame, c = '#2c7c94', r = 3)) #blue

plt.show(disp, "05 Quad With Dynamics Frame #" + str(idx))
ViewInteractiveWidget(plt.window)

(4244267, 3)
(850273, 3)
(785547, 3)


ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

# Run VICET on single frame

In [156]:
from linear_corrector import LC

A0 = np.array([0., 0., 0, 0, 0, 0,
               0, 0, 0, 0, 0, 0])
max_buffer = 0.5 

dc = LC(cloud1 = submap_in_pc1_frame, cloud2 = pc1, fid = 60, niter = 30, 
        draw = True, mnp = 50, RM = False, solver = '12_state', 
        max_buffer = max_buffer, A0 = A0)
# dc = LC(cloud1 = pc1, cloud2 = pc2_in_pc1_frame, fid = 60, niter = 50, 
#         draw = True, mnp = 50, RM = False, solver = '12_state', 
#         max_buffer = max_buffer, A0 = A0)

ViewInteractiveWidget(dc.plt.window)

A0:
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
~~~~~~~~~~~Iteration  0 ~~~~~~~~~~
A: 
 [-0.0132  0.0442  0.0204  0.0012 -0.0005 -0.0064] 
 [ 0.0039  0.0022 -0.0586 -0.0069  0.0037  0.0005]
~~~~~~~~~~~Iteration  1 ~~~~~~~~~~
A: 
 [-0.0219  0.0771  0.039   0.0025 -0.0007 -0.0116] 
 [ 0.004   0.0043 -0.1141 -0.0134  0.0069  0.001 ]
~~~~~~~~~~~Iteration  2 ~~~~~~~~~~
A: 
 [-0.0282  0.1023  0.0566  0.0039 -0.0008 -0.0158] 
 [ 0.0009  0.0052 -0.1718 -0.0194  0.0098  0.0011]
~~~~~~~~~~~Iteration  3 ~~~~~~~~~~
A: 
 [-0.0301  0.1218  0.071   0.0048 -0.0011 -0.0192] 
 [-0.0075  0.005  -0.2194 -0.0232  0.0128  0.0012]
~~~~~~~~~~~Iteration  4 ~~~~~~~~~~
A: 
 [-0.0292  0.1361  0.0822  0.0057 -0.0016 -0.022 ] 
 [-0.0148  0.0046 -0.2592 -0.0265  0.016   0.0011]
~~~~~~~~~~~Iteration  5 ~~~~~~~~~~
A: 
 [-0.0258  0.1468  0.0888  0.0067 -0.0021 -0.0241] 
 [-0.0223  0.0041 -0.2878 -0.0294  0.0188  0.001 ]
~~~~~~~~~~~Iteration  6 ~~~~~~~~~~
A: 
 [-0.0197  0.1548  0.0931  0.0074 -0.0031 -0.0258] 
 [-0.0346  0.0

ViewInteractiveWidget(height=1376, layout=Layout(height='auto', width='100%'), width=2488)

In [ ]:
#plot velocity of platform...
from matplotlib import pyplot as p

print(gt_lidarframe)

fig, ax = p.subplots()
# ax.plot(np.diff(poses[:,1,-1]))
ax.plot(gt_lidarframe[:,0])
